[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/python35/IINTS-SDK/blob/main/examples/notebooks/00_Quickstart.ipynb)


In [ ]:
import os
import sys
from pathlib import Path
import subprocess

def _ensure_repo():
    repo_root = Path().resolve()
    src_path = repo_root / 'src'
    if src_path.exists():
        sys.path.insert(0, str(src_path))
        return repo_root
    repo_dir = Path('IINTS-SDK')
    if not repo_dir.exists():
        subprocess.check_call(['git', 'clone', 'https://github.com/python35/IINTS-SDK.git'])
    repo_root = repo_dir.resolve()
    os.chdir(repo_root)
    src_path = repo_root / 'src'
    sys.path.insert(0, str(src_path))
    return repo_root

_ensure_repo()


# IINTS-AF SDK Quickstart

This notebook gets you from zero to a full simulation run, including audit + PDF report.


## Setup

If you are running locally in the repo:

```bash
python3 -m pip install -e .
```


In [ ]:
import json
from pathlib import Path

import iints
from iints.core.algorithms.pid_controller import PIDController

presets = json.loads(Path("src/iints/presets/presets.json").read_text())
preset = next(p for p in presets if p["name"] == "baseline_t1d")

scenario = preset["scenario"]
patient_config = preset["patient_config"]
duration = preset["duration_minutes"]
time_step = preset["time_step_minutes"]

outputs = iints.run_simulation(
    algorithm=PIDController,
    scenario=scenario,
    patient_config=patient_config,
    duration_minutes=duration,
    time_step=time_step,
    seed=42,
    output_dir="results/quickstart",
)

outputs.keys()


## Quick Metrics Summary


In [ ]:
from iints.metrics import calculate_full_metrics

metrics = calculate_full_metrics(outputs["results"]["glucose_actual_mgdl"], duration_hours=duration/60)
metrics


## Quickstart PDF Export


In [ ]:
quickstart_pdf = iints.generate_quickstart_report(
    outputs["results"],
    "results/quickstart/quickstart_report.pdf",
    outputs["safety_report"],
)
quickstart_pdf


## Maker Faire Demo PDF


In [ ]:
demo_pdf = iints.generate_demo_report(
    outputs["results"],
    "results/quickstart/demo_report.pdf",
    outputs["safety_report"],
)
demo_pdf


In [ ]:
outputs["results"].head()


In [ ]:
outputs["safety_report"]


In [ ]:
outputs.get("report_pdf")
